### Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится organic
- для источников paid и email из России - ставим ad
- для источников paid и email не из России - ставим other
- все остальные варианты берем из traffic_source без изменений

In [2]:
import pandas as pd

In [18]:
log = pd.read_csv('visit_log.csv', sep =';')
log.traffic_source.value_counts()

direct    7013
email     3875
yandex    2900
paid      2643
google    2507
Name: traffic_source, dtype: int64

In [19]:
def source_type_func (row):
    if row['traffic_source'] == 'yandex' or row['traffic_source'] == 'google':
        source_type = 'organic'
    elif (row['traffic_source'] == 'paid' or row['traffic_source'] == 'email') and row['region'] == 'Russia':
        source_type = 'ad'
    elif (row['traffic_source'] == 'paid' or row['traffic_source'] == 'email') and row['region'] != 'Russia':
        source_type = 'other'
    else:
        source_type = row['traffic_source']
    return source_type

log['source_type'] = log.apply(source_type_func, axis = 1)

In [20]:
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


### Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

- Прочитайте содержимое файла с датафрейм
- Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [22]:
data = pd.read_csv('URLs.txt')

In [24]:
 data[data.url.str.contains(r'/[0-9]{8}-')] 

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


### Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [84]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [85]:
ratings.tail()

,userId,movieId,rating,timestamp
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663
100003,671,6565,3.5,1074784724


In [63]:
# itog = ratings.pivot_table(index='userId',values = 'timestamp', 
#                     aggfunc=['count', 'min', 'max']).reset_index()
# itog.columns = ['userId', 'count_', 'min_', 'max_']

In [113]:
itog = ratings.groupby('userId').agg(['count','min','max'])['timestamp'].reset_index()
itog.columns = ['userId', 'count_', 'min_', 'max_']
itog['life'] = itog['max_'] - itog['min_']

In [114]:
itog_days = itog [itog.count_ > 100 ]['life'].mean() / 60 / 60 / 24

In [115]:
itog_days

463.89476214829165

### Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

In [121]:
# Дана статистика услуг перевозок клиентов компании по типам:

# rzd - железнодорожные перевозки
# auto - автомобильные перевозки
# air - воздушные перевозки
# client_base - адреса клиентов
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd


,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [118]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto


,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [119]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air


,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [120]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [141]:
all_revenue = pd.concat([auto,air,rzd]).groupby('client_id').agg('sum').reset_index()

In [142]:
all_revenue.head()

,client_id,auto_revenue,air_revenue,rzd_revenue
0,111,0.0,0.0,1093.0
1,112,0.0,0.0,2810.0
2,113,57483.0,0.0,10283.0
3,114,83.0,0.0,5774.0
4,115,912.0,81.0,981.0


In [138]:
all_revenue.merge(client_base, how='left' , on='client_id').head(10)

,client_id,auto_revenue,air_revenue,rzd_revenue,address
0,111,0.0,0.0,1093.0,Комсомольская 4
1,112,0.0,0.0,2810.0,Энтузиастов 8а
2,113,57483.0,0.0,10283.0,Левобережная 1а
3,114,83.0,0.0,5774.0,Мира 14
4,115,912.0,81.0,981.0,ЗЖБИиДК 1
5,116,4834.0,4.0,0.0,Строителей 18
6,117,98.0,13.0,0.0,Панфиловская 33
7,118,0.0,173.0,0.0,Мастеркова 4
